In [12]:
import sys
import json
import numpy as np
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [13]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [14]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [15]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [16]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())

# inputs

In [17]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

zones: 100%|██████████| 25/25 [00:01<00:00, 22.88it/s]          


# pathfinder

In [18]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set= None,
    engine='b', # b is faster!
)


start publicpathfinder
path_analysis


In [19]:

sm.step_road_pathfinder(method='aon', access_time='time',path_analysis=False)

self.volumes does not exist. od generated with self.zones


In [20]:
sm.pt_los

,origin,destination,gtime,path,pathfinder_session,reversed
0,zone_0,zone_0,0.000000,"(zone_0,)",best_path,False
1,zone_0,zone_1,12078.042640,"(zone_0, 1945, link_7313, link_7314, link_7315...",best_path,False
2,zone_0,zone_10,18365.817067,"(zone_0, 1945, link_7313, link_7314, link_7315...",best_path,False
3,zone_0,zone_100,26982.258048,"(zone_0, 1945, link_7313, link_7314, link_7315...",best_path,False
4,zone_0,zone_101,62526.973599,"(zone_0, 1945, link_7313, link_7314, link_7315...",best_path,False
...,...,...,...,...,...,...
249995,zone_99,zone_95,31275.236947,"(zone_99, road_node_71221, road_node_40999, ro...",best_path,False
249996,zone_99,zone_96,45581.963748,"(zone_99, road_node_5397, road_node_89038, roa...",best_path,False
249997,zone_99,zone_97,16855.478649,"(zone_99, road_node_71221, road_node_40999, ro...",best_path,False
249998,zone_99,zone_98,90479.638781,"(zone_99, road_node_5397, road_node_89038, roa...",best_path,False


In [21]:
sm.car_los.drop('path', axis=1, inplace=True)
sm.pt_los.drop('path', axis=1, inplace=True)

In [22]:
sm.to_zippedpickles(model_folder +'aon_los', only_attributes=['car_los','pt_los','zones'])

car_los: 100%|██████████| 28/28 [00:00<00:00, 33.01it/s]
